In [1]:
import pandas as pd 
import umap
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
import plotly.express as px
from functions.common_functions import *
import rdkit

/Users/macbook/miniconda3/envs/deepchem-test/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/macbook/miniconda3/envs/deepchem-test/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/macbook/miniconda3/envs/deepchem-test/lib/p

In [2]:
df = pd.read_csv('../data/processed/mannich_reaction_descriptors.csv')

vt = VarianceThreshold()

parameters = df.iloc[:,4:] ## Set parameters to desired columns ##
thres = vt.fit(parameters)
parameters = thres.transform(parameters)

num_reactions = len(df)
scaled_data = StandardScaler().fit_transform(parameters)

reducer = umap.UMAP(
    random_state=25,
    min_dist=0,  # Default is 0.1
    n_neighbors=num_reactions-1,  # Default is 15
    n_components=10)

clustering_embedding = reducer.fit_transform(scaled_data)

range_, inertia = get_clusters(clustering_embedding,50)

labels = k_cluster(clustering_embedding,11)
rxn_cluster_df = pd.DataFrame({'Reaction':df['Reaction'],'cluster_label':labels})

full_df = pd.read_csv('../data/processed/mannich_database_processed.csv')
full_df_trim = full_df.loc[:,['Reaction','Catalyst_Type','ee']]
full_df_clusters = pd.merge(full_df_trim,rxn_cluster_df,how='inner',on='Reaction')

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [118]:
import rdkit
from rdkit.Chem import AllChem

my_elements = {6: "C", 8: "O", 1: "H", 16: "S", 7: "N", 15: "P", 9:"F", 14: "Si", 17:"Cl"}


def mol_to_grid(mol):
    coordinates = []
    IDs = []
    stat=AllChem.EmbedMolecule(mol)
    if stat==-1:
        print("Wrong conf!!")

        return coordinates,IDs
        
    else:
        for i, atom in enumerate(mol.GetAtoms()):
            atom_ID = atom.GetIdx()
            conf = mol.GetConformer(0).GetAtomPosition(i)
            position_x = float(conf[0])
            position_y = float(conf[1])
            position_z = float(conf[2])
            coordinates.append([position_x,position_y,position_z])
            IDs.append(int(atom_ID))
        return coordinates,IDs

def find_candidates(mol,atom_type="N",bond_type="DOUBLE"):
        atom_IDs = []
        for i, atom in enumerate(mol.GetAtoms()):
            if str(atom.GetSymbol()) == "N" and int(atom.GetDegree()) == 2:
                atom_IDs.append(int(atom.GetIdx())+1)
        return atom_IDs      

def make_grid(sml, find_cand=False, scale_factor=10,add_H=True):
    
    mol = Chem.MolFromSmiles(sml)
    if add_H:
        mol = Chem.AddHs(mol)
    G, IDs  = mol_to_grid(mol)
    if len(G)==0:
        empty_grid=[]
        ID_grid=[] 
        adj=[]
        return  empty_grid, ID_grid, adj
    adj = Chem.rdmolops.GetAdjacencyMatrix(mol)
    G = np.array(G)
    
    if np.min(G) < 0:
        corec_vec = [-np.min(G), -np.min(G),-np.min(G)]
        G += corec_vec
    G=G*scale_factor
    grid_shape = (int(np.max(G)+3), int(np.max(G)+3), int(np.max(G)+3))

    empty_grid = np.zeros(grid_shape, dtype=int)
    ID_grid = np.zeros(grid_shape, dtype=int)

    for i, obj_coords in enumerate(G):
        x, y, z = obj_coords
        x_idx = int(x)  
        y_idx = int(y)
        z_idx = int(z)
        
        empty_grid[x_idx, y_idx, z_idx] = 1
        ID_grid[x_idx, y_idx, z_idx] = IDs[i]+1
    if find_cand:
        cand_ids = find_candidates(mol,atom_type="N",bond_type="DOUBLE")
        return empty_grid, ID_grid, adj, cand_ids
    else:
        return empty_grid, ID_grid, adj


In [119]:
cat_grid = []
labels = []
ID_cat_grid = []
ID_sub_grid = []
adjeciency_mat_sub = []
adjeciency_mat_cat = []
candidates = []
cat_uniq = np.unique(full_df.Catalyst)
for i in range(len(cat_uniq)):
        #try:        
                emp, ID, adj = make_grid(cat_uniq[i], find_cand=False, scale_factor=10,add_H=False)  
                cat_grid.append(emp)
                ID_cat_grid.append(ID)
                adjeciency_mat_cat.append(adj)
                
        

sub_uniq = np.unique(full_df.Imine_SMILES)
sub_grid = []

for i in range(len(sub_uniq)):
        emp, ID, adj, cand_ids = make_grid(sub_uniq[i], find_cand=True)
        sub_grid.append(emp)
        ID_sub_grid.append(ID)
        adjeciency_mat_sub.append(adj)
        candidates.append(cand_ids)



[14:01:56] Molecule does not have explicit Hs. Consider calling AddHs()
[14:01:56] Molecule does not have explicit Hs. Consider calling AddHs()
[14:01:56] Molecule does not have explicit Hs. Consider calling AddHs()
[14:01:56] Molecule does not have explicit Hs. Consider calling AddHs()
[14:01:57] Molecule does not have explicit Hs. Consider calling AddHs()
[14:01:57] Molecule does not have explicit Hs. Consider calling AddHs()
[14:01:57] Molecule does not have explicit Hs. Consider calling AddHs()
[14:01:57] Molecule does not have explicit Hs. Consider calling AddHs()
[14:01:57] Molecule does not have explicit Hs. Consider calling AddHs()
[14:01:57] Molecule does not have explicit Hs. Consider calling AddHs()
[14:01:57] Molecule does not have explicit Hs. Consider calling AddHs()
[14:01:57] Molecule does not have explicit Hs. Consider calling AddHs()
[14:01:57] Molecule does not have explicit Hs. Consider calling AddHs()
[14:01:57] Molecule does not have explicit Hs. Consider calling 

In [6]:
import plotly.graph_objects as go
empty_grid=sub_grid[0]
def plot_empty_grid(empty_grid):
    x_coords, y_coords, z_coords = np.where(empty_grid != 0)
    values = empty_grid[x_coords, y_coords, z_coords]  # Get values at each coordinate

    # Create a 3D scatter plot using Plotly with colors based on values
    trace = go.Scatter3d(
        x=x_coords,
        y=y_coords,
        z=z_coords,
        mode='markers',
        marker=dict(
            size=10,
            color=values,  # Color by values
            colorscale='Viridis',  # You can choose a different colormap
            colorbar=dict(title='Values'),
        ),
    )

    max_x, max_y, max_z = empty_grid.shape
    layout = go.Layout(
        scene=dict(
            xaxis=dict(title='X', range=[0, max_x - 1]),
            yaxis=dict(title='Y', range=[0, max_y - 1]),
            zaxis=dict(title='Z', range=[0, max_z - 1]),
        ),
        margin=dict(l=0, r=0, b=0, t=0),
    )

    fig = go.Figure(data=[trace], layout=layout)
    fig.show()
plot_empty_grid(empty_grid)

In [7]:


def find_neighbours(empty_grid, grid_shape, radius):
    second_grid = np.zeros(grid_shape, dtype=int)
      # Center coordinates
    occupied_points  = np.argwhere(empty_grid != 0)
    for point in occupied_points:
        x, y, z = point

        for dx in range(-radius, radius):
            for dy in range(-radius, radius):
                for dz in range(-radius, radius):
                    if (
                        0 <= x + dx**2 < grid_shape[0] and
                        0 <= y + dy**2 < grid_shape[1] and
                        0 <= z + dz**2 < grid_shape[2] and
                        dz**2+dy**2+dx**2 <= radius**2):
                            second_grid[x + dx, y + dy, z + dz] = 2

    return second_grid




atom_grid = find_neighbours(ID_sub_grid[4], np.shape(ID_sub_grid[4]),4)
plot_empty_grid(atom_grid)

In [8]:
from numba import jit

def calc_empty_space(cords, grid, radius):
    x = cords[0]
    y = cords[1]
    z = cords[2]
    empty_count = 0
    for dx in range(-radius, radius):
        for dy in range(-radius, radius):
            for dz in range(-radius, radius):
                new_x = x + dx
                new_y = y + dy
                new_z = z + dz

                if (
                    0 <= new_x < grid.shape[0] and
                    0 <= new_y < grid.shape[1] and
                    0 <= new_z < grid.shape[2]
                ):
                    if grid[new_x, new_y, new_z] == 0:
                        empty_count += 1

    return empty_count

def calc_occupied_space(cords, grid, radius):
    x = cords[0]
    y = cords[1]
    z = cords[2]
    occupied_count = 0
    for dx in range(-radius, radius):
        for dy in range(-radius, radius):
            for dz in range(-radius, radius):
                new_x = x + dx
                new_y = y + dy
                new_z = z + dz

                if (
                    0 <= new_x < grid.shape[0] and
                    0 <= new_y < grid.shape[1] and
                    0 <= new_z < grid.shape[2]
                ):
                    if grid[new_x, new_y, new_z] != 0:
                        occupied_count += 1

    return occupied_count

calc_empty_space([3,2,2],empty_grid, 1)
calc_occupied_space([3,2,2],empty_grid, 5)


0

In [10]:
@jit
def bresenham_line_jit(x1, y1, z1, x2, y2, z2):
    """
    Generate a list of 3D coordinates representing a line between two points using Bresenham's algorithm.

    Parameters:
        x1, y1, z1 (int): The starting coordinates.
        x2, y2, z2 (int): The ending coordinates.

    Returns:
        list: A list of 3D coordinates representing the line.
    """
    line_coords = []
    dx = abs(x2 - x1)
    dy = abs(y2 - y1)
    dz = abs(z2 - z1)
    x, y, z = x1, y1, z1
    sx = 1 if x1 < x2 else -1
    sy = 1 if y1 < y2 else -1
    sz = 1 if z1 < z2 else -1

    if dx >= dy and dx >= dz:
        err1 = 2 * dy - dx
        err2 = 2 * dz - dx
        while x != x2:
            line_coords.append((x, y, z))
            if err1 > 0:
                y += sy
                err1 -= 2 * dx
            if err2 > 0:
                z += sz
                err2 -= 2 * dx
            err1 += 2 * dy
            err2 += 2 * dz
            x += sx
    elif dy >= dx and dy >= dz:
        err1 = 2 * dx - dy
        err2 = 2 * dz - dy
        while y != y2:
            line_coords.append((x, y, z))
            if err1 > 0:
                x += sx
                err1 -= 2 * dy
            if err2 > 0:
                z += sz
                err2 -= 2 * dy
            err1 += 2 * dx
            err2 += 2 * dz
            y += sy
    else:
        err1 = 2 * dy - dz
        err2 = 2 * dx - dz
        while z != z2:
            line_coords.append((x, y, z))
            if err1 > 0:
                y += sy
                err1 -= 2 * dz
            if err2 > 0:
                x += sx
                err2 -= 2 * dz
            err1 += 2 * dy
            err2 += 2 * dx
            z += sz
    line_coords.append((x2, y2, z2))
    return line_coords
@jit
def create_grid_with_lines_jit(grid_shape, lines, radius=1):
    """
    Create a new grid with indices where lines pass through and their neighbors set to 3.

    Parameters:
        grid_shape (tuple): The shape of the grid (e.g., (5, 5, 5)).
        lines (list): A list of 3D coordinates representing lines.

    Returns:
        numpy.ndarray: The new grid with values indicating line positions and their neighbors.
    """
    new_grid = np.zeros(grid_shape, dtype=int)
    for bond in lines:
        bonds = bond
        for line in bonds:
            for x, y, z in line:
                new_grid[x, y, z] = 3  # Set the line position to 2

                # Update neighboring cells to 3
                for dx in range(-radius, radius):
                    for dy in range(-radius, radius):
                        for dz in range(-radius, radius):
                            if (
                                0 <= x + dx**2 < grid_shape[0] and
                                0 <= y + dy**2 < grid_shape[1] and
                                0 <= z + dz**2 < grid_shape[2] and
                                dz**2+dy**2+dx**2 <= radius**2):
                                    new_grid[x + dx, y + dy, z + dz] = 3

    return new_grid



# Generate the coordinates for the line



/var/folders/bg/0172ww9j3x9_mqwpf6wh9jj80000gn/T/ipykernel_58791/1158466827.py:2: NumbaDeprecationWarning:

The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.

/var/folders/bg/0172ww9j3x9_mqwpf6wh9jj80000gn/T/ipykernel_58791/1158466827.py:67: NumbaDeprecationWarning:

The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.



In [11]:

def bresenham_line(x1, y1, z1, x2, y2, z2):
    """
    Generate a list of 3D coordinates representing a line between two points using Bresenham's algorithm.

    Parameters:
        x1, y1, z1 (int): The starting coordinates.
        x2, y2, z2 (int): The ending coordinates.

    Returns:
        list: A list of 3D coordinates representing the line.
    """
    line_coords = []
    dx = abs(x2 - x1)
    dy = abs(y2 - y1)
    dz = abs(z2 - z1)
    x, y, z = x1, y1, z1
    sx = 1 if x1 < x2 else -1
    sy = 1 if y1 < y2 else -1
    sz = 1 if z1 < z2 else -1

    if dx >= dy and dx >= dz:
        err1 = 2 * dy - dx
        err2 = 2 * dz - dx
        while x != x2:
            line_coords.append((x, y, z))
            if err1 > 0:
                y += sy
                err1 -= 2 * dx
            if err2 > 0:
                z += sz
                err2 -= 2 * dx
            err1 += 2 * dy
            err2 += 2 * dz
            x += sx
    elif dy >= dx and dy >= dz:
        err1 = 2 * dx - dy
        err2 = 2 * dz - dy
        while y != y2:
            line_coords.append((x, y, z))
            if err1 > 0:
                x += sx
                err1 -= 2 * dy
            if err2 > 0:
                z += sz
                err2 -= 2 * dy
            err1 += 2 * dx
            err2 += 2 * dz
            y += sy
    else:
        err1 = 2 * dy - dz
        err2 = 2 * dx - dz
        while z != z2:
            line_coords.append((x, y, z))
            if err1 > 0:
                y += sy
                err1 -= 2 * dz
            if err2 > 0:
                x += sx
                err2 -= 2 * dz
            err1 += 2 * dy
            err2 += 2 * dx
            z += sz
    line_coords.append((x2, y2, z2))
    return line_coords

def create_grid_with_lines(grid_shape, lines, radius=1):
    """
    Create a new grid with indices where lines pass through and their neighbors set to 3.

    Parameters:
        grid_shape (tuple): The shape of the grid (e.g., (5, 5, 5)).
        lines (list): A list of 3D coordinates representing lines.

    Returns:
        numpy.ndarray: The new grid with values indicating line positions and their neighbors.
    """
    new_grid = np.zeros(grid_shape, dtype=int)
    for bond in lines:
        bonds = bond
        for line in bonds:
            for x, y, z in line:
                new_grid[x, y, z] = 3  # Set the line position to 2

                # Update neighboring cells to 3
                for dx in range(-radius, radius):
                    for dy in range(-radius, radius):
                        for dz in range(-radius, radius):
                            if (
                                0 <= x + dx**2 < grid_shape[0] and
                                0 <= y + dy**2 < grid_shape[1] and
                                0 <= z + dz**2 < grid_shape[2] and
                                dz**2+dy**2+dx**2 <= radius**2):
                                    new_grid[x + dx, y + dy, z + dz] = 3

    return new_grid



# Generate the coordinates for the line



In [12]:
@jit(forceobj=True)
def create_lines_from_adjacency_matrix_jit(matrix_ids, adjacency_matrix,plot=False,radius=1):
    """
    Create lines between objects based on an adjacency matrix and positions in the empty grid.

    Parameters:
        empty_grid (numpy.ndarray): The 3D grid with occupied and empty points.
        adjacency_matrix (numpy.ndarray): The adjacency matrix representing connections between objects.

    Returns:
        plotly.graph_objs.Scatter3d: A Plotly Scatter3d object representing the lines between connected objects.
    """
    lines = []
    line_coords =[]
    num_objects = np.max(matrix_ids)
    # Get the coordinates of occupied points and their corresponding IDs
    occupied_points  = np.where(matrix_ids != 0)
    occupied_ids = matrix_ids[occupied_points]

    for i in range(num_objects):
        for j in range(i , num_objects):
            if adjacency_matrix[i, j] == 1:
                # Find the positions of the connected objects
                obj1_positions = np.argwhere(matrix_ids == j+1)
                obj2_positions = np.argwhere(matrix_ids == i+1)

                # Create lines between connected objects
                for pos1 in obj1_positions:
                    for pos2 in obj2_positions:
                        line_coords.append(bresenham_line_jit(pos1[0], pos1[1], pos1[2], pos2[0], pos2[1], pos2[2]))
    new_grid = create_grid_with_lines_jit(np.shape(matrix_ids), [line_coords],radius)


    if plot==True:
                            line = go.Scatter3d(
                                x=[pos1[0], pos2[0]],
                                y=[pos1[1], pos2[1]],
                                z=[pos1[2], pos2[2]],
                                mode='lines',
                                line=dict(width=2, color='blue'),  # Customize line properties
                            )
                            lines.append(line)
                            return new_grid, lines

    return new_grid,line_coords






# Create lines based on the adjacency matrix
lines, lc = create_lines_from_adjacency_matrix_jit(ID_sub_grid[4],adjeciency_mat_sub[4])



/var/folders/bg/0172ww9j3x9_mqwpf6wh9jj80000gn/T/ipykernel_58791/1158466827.py:66: NumbaWarning:


Compilation is falling back to object mode WITH looplifting enabled because Function "create_grid_with_lines_jit" failed type inference due to: No implementation of function Function(<built-in function zeros>) found for signature:
 
 >>> zeros(UniTuple(int64 x 3), dtype=Function(<class 'int'>))
 
There are 2 candidate implementations:
  - Of which 2 did not match due to:
  Overload in function 'ol_np_zeros': File: numba/np/arrayobj.py: Line 4317.
    With argument(s): '(UniTuple(int64 x 3), dtype=Function(<class 'int'>))':
   Rejected as the implementation raised a specific error:
     TypingError: Failed in nopython mode pipeline (step: nopython frontend)
   No implementation of function Function(<built-in function empty>) found for signature:
    
    >>> empty(UniTuple(int64 x 3), dtype=Function(<class 'int'>))
    
   There are 2 candidate implementations:
         - Of which 2 did no

CompilerError: Failed in object mode pipeline (step: remove phis nodes)
[1mIllegal IR, del found at: del $108for_iter.8
[1m
File "../../../../../var/folders/bg/0172ww9j3x9_mqwpf6wh9jj80000gn/T/ipykernel_58791/1158466827.py", line 88:[0m
[1m<source missing, REPL/exec in use?>[0m
[0m

In [13]:
def create_lines_from_adjacency_matrix(matrix_ids, adjacency_matrix,plot=False,radius=1):
    """
    Create lines between objects based on an adjacency matrix and positions in the empty grid.

    Parameters:
        empty_grid (numpy.ndarray): The 3D grid with occupied and empty points.
        adjacency_matrix (numpy.ndarray): The adjacency matrix representing connections between objects.

    Returns:
        plotly.graph_objs.Scatter3d: A Plotly Scatter3d object representing the lines between connected objects.
    """
    lines = []
    line_coords =[]
    num_objects = np.max(matrix_ids)
    # Get the coordinates of occupied points and their corresponding IDs
    occupied_points  = np.where(matrix_ids != 0)
    occupied_ids = matrix_ids[occupied_points]

    for i in range(num_objects):
        for j in range(i , num_objects):
            if adjacency_matrix[i, j] == 1:
                # Find the positions of the connected objects
                obj1_positions = np.argwhere(matrix_ids == j+1)
                obj2_positions = np.argwhere(matrix_ids == i+1)

                # Create lines between connected objects
                for pos1 in obj1_positions:
                    for pos2 in obj2_positions:
                        line_coords.append(bresenham_line(pos1[0], pos1[1], pos1[2], pos2[0], pos2[1], pos2[2]))
    new_grid = create_grid_with_lines(np.shape(matrix_ids), [line_coords],radius)


    if plot==True:
                            line = go.Scatter3d(
                                x=[pos1[0], pos2[0]],
                                y=[pos1[1], pos2[1]],
                                z=[pos1[2], pos2[2]],
                                mode='lines',
                                line=dict(width=2, color='blue'),  # Customize line properties
                            )
                            lines.append(line)
                            return new_grid, lines

    return new_grid,line_coords

In [16]:
for i in range(100):
    lines, lc = create_lines_from_adjacency_matrix(ID_sub_grid[4],adjeciency_mat_sub[4])


In [715]:
for i in range(100):
    lines, lc = create_lines_from_adjacency_matrix_jit(ID_sub_grid[4],adjeciency_mat_sub[4])


/Users/macbook/miniconda3/envs/deepchem-test/lib/python3.10/site-packages/numba/core/ir_utils.py:2149: NumbaPendingDeprecationWarning:


Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'lines' of function 'create_grid_with_lines_jit'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "../../../../../var/folders/bg/0172ww9j3x9_mqwpf6wh9jj80000gn/T/ipykernel_76524/1158466827.py", line 79:
<source missing, REPL/exec in use?>


/var/folders/bg/0172ww9j3x9_mqwpf6wh9jj80000gn/T/ipykernel_76524/1158466827.py:66: NumbaWarning:


Compilation is falling back to object mode WITHOUT looplifting enabled because Function create_grid_with_lines_jit failed at nopython mode lowering due to: cannot reflect element of reflected container: reflected list(reflected list(reflected list(UniTuple(int64 x 3))<iv=None>)<iv=None>)<iv=None>




CompilerError: Failed in object mode pipeline (step: remove phis nodes)
[1mIllegal IR, del found at: del $108for_iter.8
[1m
File "../../../../../var/folders/bg/0172ww9j3x9_mqwpf6wh9jj80000gn/T/ipykernel_76524/1158466827.py", line 88:[0m
[1m<source missing, REPL/exec in use?>[0m
[0m

In [14]:
def get_unoccupied_shape(cords, grid, radius):
    shape_grid = np.zeros(np.shape(grid), dtype=int)
    
    x = cords[0]
    y = cords[1]
    z = cords[2]
    for dx in range(-radius, radius):
        for dy in range(-radius, radius):
            for dz in range(-radius, radius):
                new_x = x + dx
                new_y = y + dy
                new_z = z + dz

                if (
                    0 <= new_x < grid.shape[0] and
                    0 <= new_y < grid.shape[1] and
                    0 <= new_z < grid.shape[2]
                ):
                    if grid[new_x, new_y, new_z] == 0:
                        shape_grid[new_x, new_y, new_z]=10

    return shape_grid

def get_unoccupied_shape_small(cords, grid, radius):
        # Get the unoccupied shape around the specified point within radius r
    unoccupied_shape = np.zeros((2*radius+1, 2*radius+1, 2*radius+1), dtype=int)
     
    x = cords[0]
    y = cords[1]
    z = cords[2]
    for dx in range(-radius, radius):
        for dy in range(-radius, radius):
            for dz in range(-radius, radius):
                new_x = x + dx
                new_y = y + dy
                new_z = z + dz

                if (
                    0 <= new_x < grid.shape[0] and
                    0 <= new_y < grid.shape[1] and
                    0 <= new_z < grid.shape[2]
                ):
                    if grid[new_x, new_y, new_z] == 0:
                        unoccupied_shape[ radius + dx, radius + dy, radius + dz]=10
                        

    return unoccupied_shape

In [17]:
def calc_candidates_occupancy(matrix_ids: np.array,  target_ids: list, lines, r:int = 2, shape_return=False):
    """
    Find all points in the grid with specified IDs.

    Parameters:
        matrix_ids (numpy.ndarray): The 3D grid with object IDs.
        target_ids (list): A list of IDs to search for.

    Returns:
        list: A list of 3D coordinates (x, y, z) of points with matching IDs.
    """
    occupied_list = []
    occupied_shape_list = []
    for target_id in target_ids:
        # Find coordinates of points with the specified ID
        points = np.argwhere(matrix_ids == target_id)
        empty_space_ratio = calc_empty_space(points[0], lines, r)
        occupied_space_ratio = calc_occupied_space(points[0], lines, r)
        occupied_list.append(occupied_space_ratio/((2*r)**3))
        if shape_return=="small":
            occupied_shape_list.append(get_unoccupied_shape_small(points[0], lines, r))
        if shape_return=="large":
            occupied_shape_list.append(get_unoccupied_shape(points[0], lines, r))
        
        
    if shape_return is not False:

        return occupied_list,occupied_shape_list
    else:
        return occupied_list
    
occupied_list,occupied_shape_list = calc_candidates_occupancy(ID_sub_grid[4], candidates[4], lines, r = 4, shape_return="large")



In [18]:
@jit
def join_grids_jit(grid_list:list):
    """
    Join several grids into one where values are based on the grid they came from.

    Parameters:
        grid_list (list of numpy.ndarray): A list of 3D grids to be joined.

    Returns:
        numpy.ndarray: The joined grid where values are based on the original grids.
    """
    # Determine the shape of the joined grid based on the maximum dimensions of the input grids
    if isinstance(grid_list, list):
        joined_grid = np.zeros(np.shape(grid_list[0]), dtype=int)

        for i, grid in enumerate(grid_list):
            # Assign a unique value to each grid in the joined grid for non-zero cells
            joined_grid[grid != 0] = (i + 1)

        return joined_grid
    else:
        print("No grids available")



/var/folders/bg/0172ww9j3x9_mqwpf6wh9jj80000gn/T/ipykernel_58791/1318348002.py:2: NumbaDeprecationWarning:

The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.



In [19]:
def join_grids(grid_list:list):
    """
    Join several grids into one where values are based on the grid they came from.

    Parameters:
        grid_list (list of numpy.ndarray): A list of 3D grids to be joined.

    Returns:
        numpy.ndarray: The joined grid where values are based on the original grids.
    """
    # Determine the shape of the joined grid based on the maximum dimensions of the input grids
    if isinstance(grid_list, list):
        joined_grid = np.zeros(np.shape(grid_list[0]), dtype=int)

        for i, grid in enumerate(grid_list):
            # Assign a unique value to each grid in the joined grid for non-zero cells
            joined_grid[grid != 0] = (i + 1)

        return joined_grid
    else:
        print("No grids available")








In [696]:
for i in range(100):
    joined_grid = join_grids([lines, atom_grid])




In [697]:
for i in range(100):
    join_grids_jit([lines, atom_grid])

/var/folders/bg/0172ww9j3x9_mqwpf6wh9jj80000gn/T/ipykernel_76524/1318348002.py:13: NumbaExperimentalFeatureWarning:

Use of isinstance() detected. This is an experimental feature.

/var/folders/bg/0172ww9j3x9_mqwpf6wh9jj80000gn/T/ipykernel_76524/1318348002.py:13: NumbaPendingDeprecationWarning:


Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'var' of function 'ol_isinstance_no_warn.<locals>.true_impl'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "../../../miniconda3/envs/deepchem-test/lib/python3.10/site-packages/numba/cpython/builtins.py", line 764:

    def true_impl(var, typs):
    ^


/var/folders/bg/0172ww9j3x9_mqwpf6wh9jj80000gn/T/ipykernel_76524/1318348002.py:1: NumbaWarning:


Compilation is falling back to object mode WITH looplifting enabled because Function "join_grids_jit" failed type inference due to: No implem

In [20]:
def can_fit_shape_in_grid(shape, second_grid):
    """
    Check if a shape can fit into the empty space in the second grid without overlapping.

    Parameters:
        shape (numpy.ndarray): The 3D grid representing the shape (non-zero elements).
        second_grid (numpy.ndarray): The second grid where shapes should not overlap.

    Returns:
        bool: True if the shape can fit without overlapping, False otherwise.
    """
    # Find the coordinates of non-zero elements in the shape
    shape_coords = np.argwhere(shape != 0)

    # Iterate through the shape coordinates
    for coord in shape_coords:
        x, y, z = coord

        # Check if the shape would overlap with any non-zero elements in the second grid
        if second_grid[x, y, z] != 0:
            return False

    return True

occupied_shape_list[0]

can_fit_shape_in_grid(occupied_shape_list[0], occupied_shape_list[1])


True

In [23]:
@jit
def can_fit_shape_in_grid_jit(shape, second_grid):
    """
    Check if a shape can fit into the empty space in the second grid without overlapping.

    Parameters:
        shape (numpy.ndarray): The 3D grid representing the shape (non-zero elements).
        second_grid (numpy.ndarray): The second grid where shapes should not overlap.

    Returns:
        bool: True if the shape can fit without overlapping, False otherwise.
    """
    # Find the coordinates of non-zero elements in the shape
    shape_coords = np.argwhere(shape != 0)

    # Iterate through the shape coordinates
    for coord in shape_coords:
        x, y, z = coord

        # Check if the shape would overlap with any non-zero elements in the second grid
        if second_grid[x, y, z] != 0:
            return False

    return True

/var/folders/bg/0172ww9j3x9_mqwpf6wh9jj80000gn/T/ipykernel_58791/3533578163.py:2: NumbaDeprecationWarning:

The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.



In [25]:
for i in range(1000):
    can_fit_shape_in_grid(occupied_shape_list[1], occupied_shape_list[1])


In [26]:
for i in range(1000):
    can_fit_shape_in_grid_jit(occupied_shape_list[1], occupied_shape_list[1])


In [32]:
def calculate_free_space_in_octants(grid, center, radius,atomic_radius=4):
    """
    Calculate the percentage of free space in each octant around the center within a specified radius.

    Parameters:
        grid (numpy.ndarray): The 3D grid with objects (non-zero elements).
        center (tuple): The coordinates (x, y, z) of the center point.
        radius (int): The radius within which to calculate free space.

    Returns:
        list: A list containing the percentage of free space in each octant relative to the total free space in the sphere.
    """
    x_center, y_center, z_center = center
    total_free_space = 0  # Total free space within the sphere
    octant_counts = {i: 0 for i in range(8)} # Initialize percentages for each octant

    for dx in range(-radius, radius + 1):
        for dy in range(-radius, radius + 1):
            for dz in range(-radius, radius + 1):
                
                x = x_center + dx
                y = y_center + dy
                z = z_center + dz
                if (
                    0 > x or x >= np.shape(grid)[0] or
                    0 > y or y >= np.shape(grid)[1] or
                    0 > z or z >= np.shape(grid)[2]
                ):
                    total_free_space += 1
                else:
                # Check if the point is within the radius
                    if dx != 0 and dy != 0 and dz != 0:
                        if atomic_radius**2 < dx**2 + dy**2 + dz**2 <= radius**2:
                            total_free_space += 1

                            # Determine the octant based on relative position to the center
                            octant = 0
                        
                            # Check if the point is unoccupied
                            if grid[x, y, z] != 0:
                                if dx >= 0:  # Check x-coordinate
                                    octant += 1
                                if dy >= 0:  # Check y-coordinate
                                    octant += 2
                                if dz > 0:  # Check z-coordinate
                                    octant += 4
                                # Increment the count for the corresponding octant
                                octant_counts[octant] += 1

    # Calculate percentages
    free_space_percentages = {octant: (count / total_free_space) * 8*100 for octant, count in octant_counts.items()}

    return free_space_percentages
center = np.argwhere(ID_sub_grid[0] == 7)[0]

calculate_free_space_in_octants(sub_grid[0], center, 15,atomic_radius=4)




{0: 0.06680026720106881,
 1: 0.0,
 2: 0.0,
 3: 0.0,
 4: 0.0,
 5: 0.0,
 6: 0.06680026720106881,
 7: 0.0}

In [34]:

def place_evenly_spaced_lines(grid, center, N,atomic_radi):
    """
    Place N evenly spaced lines from the center in a 3D grid.

    Parameters:
        grid_shape (tuple): The shape of the 3D grid (x, y, z).
        center (tuple): The coordinates of the center point (x_center, y_center, z_center).
        N (int): The number of lines to place.

    Returns:
        list: A list of lines, where each line is represented as a list of grid points.
        int: The number of lines that pass through occupied space.
    """
    grid_shape = np.shape(grid)
    x_center, y_center, z_center = center
    x_size, y_size, z_size = grid_shape

    lines = []
    occupied_lines_count = 0


    # Calculate the directions of the evenly spaced lines in spherical coordinates
    theta_values = np.linspace(0, 2 * np.pi, N//2, endpoint=False)  # Azimuthal angle
    phi_values = np.linspace(0, np.pi, N//2, endpoint=False)  # Polar angle

    for theta in theta_values:
       for phi in phi_values:
        # Convert spherical coordinates to Cartesian coordinates for direction vector
        direction = np.array([
            np.sin(phi) * np.cos(theta),
            np.sin(phi) * np.sin(theta),
            np.cos(phi)
        ])

        # Initialize a line starting from the center
        half_line  = []
        current_point = np.array([x_center, y_center, z_center])
        crossed_occupied_space = False

        while all(0 <= current_point[i] < grid_shape[i] for i in range(3)):
            x, y, z = current_point.astype(int)
            half_line.append((x, y, z))

            # Check if the current point is occupied
            if grid[x, y, z] != 0:
                #if np.dot(direction, np.array([x - x_center, y - y_center, z - z_center])) < 0:
                    if (x_center-x)**2 + (y_center-y)**2 + (z_center-z)**2 > atomic_radi**2:
                        occupied_lines_count += 1
                        crossed_occupied_space = True
                        break
            

            current_point = current_point + direction

        
        lines.append((half_line, direction, crossed_occupied_space))

    return lines#, occupied_lines_count


center = np.argwhere(ID_sub_grid[10] == 7)[0]
print(center)
half_lines = place_evenly_spaced_lines(sub_grid[10], center, 10,4)

# Print the lines
for i, (half_line, direction, crossed_occupied_space) in enumerate(half_lines):
    print(f"Half-Line {i + 1}:")
    print(f"Crossed Occupied Space: {crossed_occupied_space}")
    print(half_line)


[16 66 59]
Half-Line 1:
Crossed Occupied Space: False
[(16, 66, 59), (16, 66, 60), (16, 66, 61), (16, 66, 62), (16, 66, 63), (16, 66, 64), (16, 66, 65), (16, 66, 66), (16, 66, 67), (16, 66, 68), (16, 66, 69), (16, 66, 70), (16, 66, 71), (16, 66, 72), (16, 66, 73), (16, 66, 74), (16, 66, 75), (16, 66, 76), (16, 66, 77), (16, 66, 78), (16, 66, 79), (16, 66, 80), (16, 66, 81), (16, 66, 82), (16, 66, 83), (16, 66, 84), (16, 66, 85), (16, 66, 86), (16, 66, 87), (16, 66, 88), (16, 66, 89), (16, 66, 90), (16, 66, 91), (16, 66, 92), (16, 66, 93), (16, 66, 94), (16, 66, 95), (16, 66, 96), (16, 66, 97), (16, 66, 98), (16, 66, 99), (16, 66, 100), (16, 66, 101), (16, 66, 102), (16, 66, 103), (16, 66, 104), (16, 66, 105), (16, 66, 106), (16, 66, 107), (16, 66, 108), (16, 66, 109), (16, 66, 110), (16, 66, 111), (16, 66, 112), (16, 66, 113), (16, 66, 114), (16, 66, 115)]
Half-Line 2:
Crossed Occupied Space: False
[(16, 66, 59), (16, 66, 59), (17, 66, 60), (17, 66, 61), (18, 66, 62), (18, 66, 63), (19

In [35]:
from plotly.subplots import make_subplots



# Example usage:

# Visualize the lines passing through the 3D grid using Plotly
#visualize_half_lines_with_plotly(cat_mol_grids[0], center, 10)
fig = make_subplots(rows=1, cols=1, specs=[[{'type': 'scatter3d'}]])

# Scatter plot for occupied space points in red
x_occupied, y_occupied, z_occupied = np.where(sub_grid[10] != 0)
fig.add_trace(go.Scatter3d(x=x_occupied, y=y_occupied, z=z_occupied, mode='markers', marker=dict(size=5, color='red'), name='Occupied Space'))

# Plot the evenly spaced half-lines that cross occupied space
for i, half_line in enumerate(half_lines):
    x, y, z = zip(*half_line[0])
    if half_line[2]:
        fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode='lines', line=dict(color='red'), name=f"Half-Line {i + 1}"))
    else:
        fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode='lines', line=dict(color='blue'), name=f"Half-Line {i + 1}"))
fig.update_layout(scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z'))
fig.update_layout(title='Evenly Spaced Half-Lines Crossing Occupied Space on Spherical Plane')

fig.show()

In [37]:
def calculate_percentage_no_clashes(half_lines, grid, center):
    """
    Calculate the percentage of lines that do not have any clashes with occupied points in different octants.

    Parameters:
        half_lines (list): A list of half-lines.
        grid_shape (tuple): The shape of the 3D grid (x, y, z).
        center (tuple): The coordinates of the center point (x_center, y_center, z_center).

    Returns:
        dict: A dictionary containing the percentage of lines without clashes for each octant.
    """
    grid_shape = np.shape(grid)
    x_size, y_size, z_size = grid_shape
    octant_counts = {i: 0 for i in range(8)}
    x_center, y_center, z_center = center
    
    
    for half_line in half_lines:
        if  half_line[2]:
                # Determine the octant based on the center point
                octant = 0
                if half_line[0][-1][0] >= x_center:  # Check x-coordinate
                    octant += 1
                if half_line[0][-1][1] >= y_center:  # Check y-coordinate
                    octant += 2
                if half_line[0][-1][2] >= z_center:  # Check z-coordinate
                    octant += 4

                octant_counts[octant] += 1

    total_lines = len(half_lines)/8
    percentages = {octant: (count / total_lines) * 100 for octant, count in octant_counts.items()}
    
    return percentages

calculate_percentage_no_clashes(half_lines, sub_grid[0],center)


{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0, 7: 0.0}

In [66]:


def place_evenly_spaced_lines(grid, center, N,atomic_radi):
    """
    Place N evenly spaced lines from the center in a 3D grid.

    Parameters:
        grid_shape (tuple): The shape of the 3D grid (x, y, z).
        center (tuple): The coordinates of the center point (x_center, y_center, z_center).
        N (int): The number of lines to place.

    Returns:
        list: A list of lines, where each line is represented as a list of grid points.
        int: The number of lines that pass through occupied space.
    """
    grid_shape = np.shape(grid)
    x_center, y_center, z_center = center
    x_size, y_size, z_size = grid_shape

    lines = []
    occupied_lines_count = 0


    # Calculate the directions of the evenly spaced lines in spherical coordinates
    theta_values = np.linspace(0, 2 * np.pi, N//2, endpoint=False)  # Azimuthal angle
    phi_values = np.linspace(0, np.pi, N//2, endpoint=False)  # Polar angle

    for theta in theta_values:
       for phi in phi_values:
        # Convert spherical coordinates to Cartesian coordinates for direction vector
        direction = np.array([
            np.sin(phi) * np.cos(theta),
            np.sin(phi) * np.sin(theta),
            np.cos(phi)
        ])

        # Initialize a line starting from the center
        half_line  = []
        current_point = np.array([x_center, y_center, z_center])
        crossed_occupied_space = False

        while all(0 <= current_point[i] < grid_shape[i] for i in range(3)):
            x, y, z = current_point.astype(int)
            half_line.append((x, y, z))

            # Check if the current point is occupied
            if grid[x, y, z] != 0:
                #if np.dot(direction, np.array([x - x_center, y - y_center, z - z_center])) < 0:
                    if (x_center-x)**2 + (y_center-y)**2 + (z_center-z)**2 > atomic_radi**2:
                        occupied_lines_count += 1
                        crossed_occupied_space = True
                        break
            

            current_point = current_point + direction

        
        lines.append((half_line, direction, crossed_occupied_space))

    return lines#, occupied_lines_count


center = np.argwhere(ID_sub_grid[1] == 7)[0]
print(center)
half_lines = place_evenly_spaced_lines(sub_grid[1], center, 10,4)

# Print the lines
for i, (half_line, direction, crossed_occupied_space) in enumerate(half_lines):
    print(f"Half-Line {i + 1}:")
    print(f"Crossed Occupied Space: {crossed_occupied_space}")
    print(half_line)


[68 60 65]
Half-Line 1:
Crossed Occupied Space: False
[(68, 60, 65), (68, 60, 66), (68, 60, 67), (68, 60, 68), (68, 60, 69), (68, 60, 70), (68, 60, 71), (68, 60, 72), (68, 60, 73), (68, 60, 74), (68, 60, 75), (68, 60, 76), (68, 60, 77), (68, 60, 78), (68, 60, 79), (68, 60, 80), (68, 60, 81), (68, 60, 82), (68, 60, 83), (68, 60, 84), (68, 60, 85), (68, 60, 86), (68, 60, 87), (68, 60, 88), (68, 60, 89), (68, 60, 90), (68, 60, 91), (68, 60, 92), (68, 60, 93), (68, 60, 94), (68, 60, 95), (68, 60, 96), (68, 60, 97), (68, 60, 98), (68, 60, 99), (68, 60, 100), (68, 60, 101), (68, 60, 102), (68, 60, 103), (68, 60, 104), (68, 60, 105), (68, 60, 106), (68, 60, 107), (68, 60, 108), (68, 60, 109), (68, 60, 110), (68, 60, 111), (68, 60, 112), (68, 60, 113), (68, 60, 114), (68, 60, 115), (68, 60, 116), (68, 60, 117), (68, 60, 118), (68, 60, 119), (68, 60, 120), (68, 60, 121), (68, 60, 122), (68, 60, 123)]
Half-Line 2:
Crossed Occupied Space: False
[(68, 60, 65), (68, 60, 65), (69, 60, 66), (69, 60, 

Only "easy" sollution to the fitting problem I have found is compare ratios of occpancy in the same box
Ideas:
Add different box dimensions/shapes and average out
Insted of exact shape for catalyst create rectangle that has to fit into that pocket

In [41]:
import json
def Analyze_space(df, Target="Imine_SMILES", find_cand=False, neighbour_radius=4, occupancy_radius=10, scale_factor=10 ,N_Lines=16, bond_radius =1):
    cat_grid = []
    ID_cat_grid = []
    adjeciency_mat_cat = []
    candidates = []
    mol_grids =[]
    results = []
    cat_uniq = np.unique(full_df[Target])
        
    for i in range(len(cat_uniq)):
        if find_cand:
            emp, ID, adj, cand_ids = make_grid(cat_uniq[i],find_cand,scale_factor)
            candidates.append(cand_ids) 
        else:
            emp, ID, adj = make_grid(cat_uniq[i],find_cand)       
        #cat_grid.append(emp)
        #ID_cat_grid.append(ID)
        #adjeciency_mat_cat.append(adj)
        atom_grid = find_neighbours(ID, np.shape(ID), radius=neighbour_radius)
        lines, lc = create_lines_from_adjacency_matrix(ID,adj,bond_radius)
        if find_cand:
            #occupied_list,occupied_shape_list = calc_candidates_occupancy(ID, cand_ids, lines, r = occupancy_radius, shape_return="large")
            grid_list = [lines, atom_grid]
            joined_grid = join_grids(grid_list)
            for cand in cand_ids:
                center = np.argwhere(ID == cand)[0]
                half_lines = place_evenly_spaced_lines(joined_grid, center, N_Lines, neighbour_radius)
                line_percentages = calculate_percentage_no_clashes(half_lines, joined_grid, center)
                free_space_percentages = calculate_free_space_in_octants(joined_grid, center, occupancy_radius*2)
                res = [cat_uniq[i], free_space_percentages, line_percentages]
                results.append(res)

            #grid_list.extend(occupied_shape_list)
            #joined_grid = join_grids(grid_list)
            
        else:
            joined_grid = join_grids([lines, atom_grid])
        mol_grids.append(joined_grid)

    return results #mol_grids, ID_cat_grid, cat_grid
            
results = Analyze_space(full_df, Target="Imine_SMILES", find_cand=True, neighbour_radius=4, occupancy_radius=10, scale_factor=10, N_Lines=16,bond_radius =1)

with open('out.json', 'w') as fout:
    json.dump(results, fout)
#cat_mol_grids, ID_cat_grid, cat_grid = Analyze_space(full_df, Target="Catalyst", find_cand=False, neighbour_radius=4, occupancy_radius=10)



# Conformer Generation with ETKDG




# Divide Catalyst into groups and see if they fit


In [50]:


def rotate_3d_grid(grid, rotation_matrix):
    """
    Rotate a 3D grid using a rotation matrix.

    Parameters:
        grid (numpy.ndarray): The 3D grid with objects (non-zero elements).
        rotation_matrix (numpy.ndarray): A 3x3 rotation matrix for the desired rotation.

    Returns:
        numpy.ndarray: The rotated 3D grid.
    """
    # Ensure the input grid is a numpy array
    grid = np.array(grid)

    # Get the shape of the grid
    grid_shape = grid.shape

    # Create an array to store the rotated grid
    rotated_grid = np.zeros_like(grid)

    # Loop through each grid point and apply the rotation
    for x in range(grid_shape[0]):
        for y in range(grid_shape[1]):
            for z in range(grid_shape[2]):
                # Check if the current point is occupied (non-zero)
                if grid[x, y, z] != 0:
                    # Apply the rotation matrix to the grid point
                    rotated_point = np.dot(rotation_matrix, np.array([x, y, z]))

                    # Round to the nearest integer to map to a grid cell
                    rx, ry, rz = np.round(rotated_point).astype(int)

                    # Ensure the rotated point is within the grid bounds
                    if 0 <= rx < grid_shape[0] and 0 <= ry < grid_shape[1] and 0 <= rz < grid_shape[2]:
                        rotated_grid[rx, ry, rz] = grid[x, y, z]

    return rotated_grid

    

In [153]:
# Define a rotation matrix (e.g., rotate 45 degrees around the Z-axis)
angle_degrees = 45
angle_radians = np.radians(angle_degrees)
rotation_matrix = np.array([
    [np.cos(angle_radians), -np.sin(angle_radians), 0],
    [np.sin(angle_radians), np.cos(angle_radians), 0],
    [0, 0, 1]
])

# Rotate the 3D grid
rotated_grid = rotate_3d_grid(sub_grid[1], rotation_matrix)

plot_empty_grid(sub_grid[1])
plot_empty_grid(rotated_grid)

In [247]:
import numpy as np
@jit
def place_grid_into_another(larger_grid, smaller_grid, position=(0, 0, 0)):
    """
    Place a smaller 3D grid into a larger 3D grid at the specified position.

    Parameters:
        larger_grid (numpy.ndarray): The larger 3D grid.
        smaller_grid (numpy.ndarray): The smaller 3D grid to be placed.
        position (tuple): The position (x, y, z) to place the smaller grid within the larger grid.

    Returns:
        numpy.ndarray: The modified larger 3D grid with the smaller grid placed.
    """
    # Ensure input grids are numpy arrays
    larger_grid = np.array(larger_grid)
    smaller_grid = np.array(smaller_grid)

    # Get dimensions of the larger and smaller grids
    larger_shape = larger_grid.shape
    smaller_shape = smaller_grid.shape

    # Get the position where the smaller grid will be placed
    x, y, z = position

    # Check if the placement position is within the bounds of the larger grid
    if x >= 0 and y >= 0 and z >= 0:
        # Check if the smaller grid fits entirely within the larger grid
        if (x + smaller_shape[0] <= larger_shape[0] and
            y + smaller_shape[1] <= larger_shape[1] and
            z + smaller_shape[2] <= larger_shape[2]):

            # Place the smaller grid into the larger grid at the specified position
            larger_grid[x:x+smaller_shape[0], y:y+smaller_shape[1], z:z+smaller_shape[2]] = smaller_grid
        else:
            # Extend the larger grid to accommodate the smaller grid
            new_shape = (
                max(x + smaller_shape[0], larger_shape[0]),
                max(y + smaller_shape[1], larger_shape[1]),
                max(z + smaller_shape[2], larger_shape[2])
            )
            extended_grid = np.zeros(new_shape)
            extended_grid[:larger_shape[0], :larger_shape[1], :larger_shape[2]] = larger_grid
            extended_grid[x:x+smaller_shape[0], y:y+smaller_shape[1], z:z+smaller_shape[2]] = smaller_grid
            larger_grid = extended_grid
    else:
        print("Error: Placement position is out of bounds.")

    return larger_grid






combined = place_grid_into_another(sub_grid[1], rotated_grid, position=(60, 60, 90))

plot_empty_grid(combined)


/var/folders/bg/0172ww9j3x9_mqwpf6wh9jj80000gn/T/ipykernel_58791/202815955.py:3: NumbaDeprecationWarning:

The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.

/var/folders/bg/0172ww9j3x9_mqwpf6wh9jj80000gn/T/ipykernel_58791/202815955.py:2: NumbaWarning:


Compilation is falling back to object mode WITH looplifting enabled because Function "place_grid_into_another" failed type inference due to: No implementation of function Function(<built-in function array>) found for signature:
 
 >>> array(array(int64, 3d, C))
 
There are 2 candidate implementations:
      - Of which 2 did not match due to:
      Overload in function 'impl_np_array': File: numba/np/arrayobj.py: Line 5242.
        With argument(s): '(arr

In [246]:
def find_smallest_bounding_grid(grid):
    """
    Find the smallest possible grid that includes all occupied points in the given grid.

    Parameters:
        grid (numpy.ndarray): The 3D grid with occupied points (non-zero elements).

    Returns:
        numpy.ndarray: The smallest bounding grid containing all occupied points.
    """
    # Ensure the input grid is a numpy array
    grid = np.array(grid)

    # Find the coordinates of all occupied points (non-zero elements)
    occupied_points = np.transpose(np.where(grid != 0))

    # Find the minimum and maximum coordinates of the occupied points
    min_coords = np.min(occupied_points, axis=0)
    max_coords = np.max(occupied_points, axis=0)

    # Calculate the shape of the smallest bounding grid
    bounding_grid_shape = tuple(max_coords - min_coords + 1)

    # Create the smallest bounding grid and copy the occupied points to it
    bounding_grid = np.zeros(bounding_grid_shape)
    for point in occupied_points:
        relative_coords = point - min_coords
        bounding_grid[tuple(relative_coords)] = grid[tuple(point)]

    return bounding_grid

small = find_smallest_bounding_grid(cat_grid[10])
plot_empty_grid(cat_grid[10])
plot_empty_grid(small)

In [245]:
combined = place_grid_into_another(sub_grid[1], small, position=(60, 60, 90))

plot_empty_grid(combined)


In [137]:
@jit
def check_no_overlap(grid1, grid2):
    """
    Check if two grids do not overlap when one is placed next to the other.

    Parameters:
        grid1 (numpy.ndarray): The first 3D grid.
        grid2 (numpy.ndarray): The second 3D grid to be checked.

    Returns:
        bool: True if no overlap is detected, False otherwise.
    """
    # Get the shapes of the grids
    shape1 = np.array(grid1.shape)
    shape2 = np.array(grid2.shape)

    # Iterate through possible positions for the second grid
    for x in range(shape1[0] - shape2[0] + 1):
        for y in range(shape1[1] - shape2[1] + 1):
            for z in range(shape1[2] - shape2[2] + 1):
                # Create a mask for the overlapping region
                overlap_mask = grid1[x:x+shape2[0], y:y+shape2[1], z:z+shape2[2]] & grid2

                # Check if any part of the grids overlap
                if np.any(overlap_mask):
                    return False  # Overlap detected

    return True  # No overlap detected


    


/var/folders/bg/0172ww9j3x9_mqwpf6wh9jj80000gn/T/ipykernel_58791/635982954.py:2: NumbaDeprecationWarning:

The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.



In [243]:
overlap = False
move_step = 1  # Adjust this step size for movement
moved_position = np.array(half_lines[0][0][-1])
while not overlap:
    moved_position += move_step * -half_lines[0][1].astype(int)
    moved_grid = place_grid_into_another(sub_grid[1], small, position=tuple(moved_position))
    overlap = check_no_overlap( sub_grid[1], moved_grid)

if overlap:
    print("Grids overlap at position:", moved_position)
else:
    print("Grids do not overlap along the line.")

Grids overlap at position: [ 68  60 122]


In [240]:

def place_grid_into_another(larger_grid, smaller_grid, position=(0, 0, 0)):
    """
    Place a smaller 3D grid into a larger 3D grid at the specified position.

    Parameters:
        larger_grid (numpy.ndarray): The larger 3D grid.
        smaller_grid (numpy.ndarray): The smaller 3D grid to be placed.
        position (tuple): The position (x, y, z) to place the smaller grid within the larger grid.

    Returns:
        numpy.ndarray: The modified larger 3D grid with the smaller grid placed.
    """
    # Ensure input grids are numpy arrays
    larger_grid = np.array(larger_grid)
    smaller_grid = np.array(smaller_grid)

    # Get dimensions of the larger and smaller grids
    larger_shape = larger_grid.shape
    smaller_shape = smaller_grid.shape

    # Get the position where the smaller grid will be placed
    x, y, z = position

    # Check if the placement position is within the bounds of the larger grid
    if x >= 0 and y >= 0 and z >= 0:
        # Check if the smaller grid fits entirely within the larger grid
        if (x + smaller_shape[0] <= larger_shape[0] and
            y + smaller_shape[1] <= larger_shape[1] and
            z + smaller_shape[2] <= larger_shape[2]):

            # Place the smaller grid into the larger grid at the specified position
            larger_grid[x:x+smaller_shape[0], y:y+smaller_shape[1], z:z+smaller_shape[2]] = smaller_grid
        else:
            # Extend the larger grid to accommodate the smaller grid
            new_shape = (
                max(x + smaller_shape[0], larger_shape[0]),
                max(y + smaller_shape[1], larger_shape[1]),
                max(z + smaller_shape[2], larger_shape[2])
            )
            extended_grid = np.zeros(new_shape)
            extended_grid[:larger_shape[0], :larger_shape[1], :larger_shape[2]] = larger_grid
            extended_grid[x:x+smaller_shape[0], y:y+smaller_shape[1], z:z+smaller_shape[2]] = smaller_grid
            larger_grid = extended_grid
    else:
        print("Error: Placement position is out of bounds.")

    return larger_grid



def find_last_non_overlap_position(sub_grid, small, line,move_step=1, max_step=1000):
    overlap = False
    step = 0
    moved_position = np.array(line[0][-1])
    while not overlap and step <max_step:
        step+=1
        moved_position += move_step * -line[1].astype(int)
        moved_grid = place_grid_into_another(sub_grid, small, position=tuple(moved_position))
        overlap = check_no_overlap(sub_grid, moved_grid)

    if overlap:
        return place_grid_into_another(sub_grid, small, position=tuple(moved_position)), overlap
    else:
        return moved_position,overlap
    

overlap_shape, overlap  = find_last_non_overlap_position(sub_grid[1], small, half_lines[0],move_step=100)
plot_empty_grid(overlap_shape)


In [242]:
search_line=0
passed_line=0

for line in half_lines:
    if not line[2]:
        search_line+=1
        if find_last_non_overlap_position(cat_grid[1], small,line,move_step=1)[1]:
            passed_line+=1

print(passed_line, search_line)

25 25


In [ ]:

def create_subgrid(starting_point, dimensions):
    """
    Create a 3D subgrid in the shape of a rectangular box with the starting point in the middle of one wall.

    Parameters:
        starting_point (tuple): The (x, y, z) coordinates of the starting point.
        dimensions (tuple): The dimensions of the rectangular box (length, width, height).

    Returns:
        numpy.ndarray: The 3D subgrid.
    """
    # Extract starting coordinates and dimensions
    x, y, z = starting_point
    length, width, height = dimensions

    # Create an empty subgrid with the specified dimensions
    subgrid = np.zeros((length, width, height))
    print(length, width, height)
    # Calculate the coordinates of the middle of one wall
    middle_x = x + length // 2
    middle_y = y + width // 2
    middle_z = z + height // 2

    # Place the subgrid at the middle of one wall
    subgrid[x:x+length-1, y:y + width-1, z:z + height-1] = 1

    return subgrid

subgrid = create_subgrid((0,0,0), np.shape(small))
rectangle = place_grid_into_another(cat_grid[10], subgrid, position=(30,61,56))
plot_empty_grid(rectangle)



In [208]:
def extract_unoccupied_shape(larger_grid, smaller_grid, position):
    """
    Extract the unoccupied shape within a subgrid after it has been placed into another grid.

    Parameters:
        larger_grid (numpy.ndarray): The larger 3D grid with the subgrid placed.
        smaller_grid (numpy.ndarray): The smaller 3D subgrid.
        position (tuple): The position (x, y, z) where the subgrid was placed.

    Returns:
        numpy.ndarray: The unoccupied shape within the subgrid.
    """
    # Ensure input grids are numpy arrays
    larger_grid = np.array(larger_grid)
    smaller_grid = np.array(smaller_grid)

    # Get the dimensions of the subgrid
    subgrid_shape = smaller_grid.shape

    # Get the position where the subgrid was placed
    x, y, z = position

    # Extract the unoccupied shape within the subgrid
    unoccupied_shape = smaller_grid.copy()
    unoccupied_shape[smaller_grid == 1] = larger_grid[x:x+subgrid_shape[0], y:y+subgrid_shape[1], z:z+subgrid_shape[2]][smaller_grid == 1]

    return unoccupied_shape

unocupied = extract_unoccupied_shape(rectangle, subgrid, position=(30,61,56))
print(np.shape(cat_grid[10]))

print(np.shape(unocupied))
print(np.shape(small))
can_fit_shape_in_grid(unocupied, small)
#print(len(np.argwhere(unocupied!=0)))
#plot_empty_grid(unocupied)


(124, 124, 124)
(122, 63, 65)
(122, 63, 65)


False

In [206]:
def can_fit_shape_in_unoccupied_shape(target_shape, unoccupied_shape):
    """
    Check if a 3D grid shape can fit into an unoccupied shape.

    Parameters:
        target_shape (numpy.ndarray): The 3D grid shape to be placed.
        unoccupied_shape (numpy.ndarray): The unoccupied shape where the target shape should fit.

    Returns:
        bool: True if the target shape can fit, False otherwise.
    """
    # Get the dimensions of the target shape and unoccupied shape
    target_shape_dimensions = target_shape.shape
    unoccupied_shape_dimensions = unoccupied_shape.shape

    # Check if the dimensions of the target shape are smaller or equal to the dimensions of the unoccupied shape
    can_fit = all(target_shape_dimensions[i] <= unoccupied_shape_dimensions[i] for i in range(3))

    return can_fit

can_fit_shape_in_unoccupied_shape(small, unocupied)

True

In [244]:
def add_layer_to_occupied_neighbors(grid, occupied_value, layer_value):
    """
    Add a layer to a 3D grid at positions where neighbors are occupied.

    Parameters:
        grid (numpy.ndarray): The 3D grid to which the layer will be added.
        occupied_value: The value that indicates an occupied position in the grid.
        layer_value: The value to set at positions where neighbors are occupied.

    Returns:
        numpy.ndarray: The modified 3D grid with the added layer.
    """
    grid_shape = grid.shape
    new_grid = np.copy(grid)  # Create a copy to modify

    for x in range(grid_shape[0]):
        for y in range(grid_shape[1]):
            for z in range(grid_shape[2]):
                # Check if the current position is occupied
                if grid[x, y, z] == occupied_value:
                    # Check and update neighbors in a 3x3x3 cube around the current position
                    for dx in [-1, 0, 1]:
                        for dy in [-1, 0, 1]:
                            for dz in [-1, 0, 1]:
                                new_x, new_y, new_z = x + dx, y + dy, z + dz
                                if (
                                    0 <= new_x < grid_shape[0] and
                                    0 <= new_y < grid_shape[1] and
                                    0 <= new_z < grid_shape[2] and
                                    grid[new_x, new_y, new_z] == 0
                                ):
                                    new_grid[new_x, new_y, new_z] = layer_value

    return new_grid
layer=small
for i in range(4):
    layer = add_layer_to_occupied_neighbors(layer, 1, 1)
plot_empty_grid(layer)

In [219]:
def add_layer_to_directly_occupied_neighbors(grid, occupied_value, layer_value):
    """
    Add a layer to a 3D grid at positions where direct neighbors are occupied.

    Parameters:
        grid (numpy.ndarray): The 3D grid to which the layer will be added.
        occupied_value: The value that indicates an occupied position in the grid.
        layer_value: The value to set at positions where direct neighbors are occupied.

    Returns:
        numpy.ndarray: The modified 3D grid with the added layer.
    """
    grid_shape = grid.shape
    new_grid = np.copy(grid)  # Create a copy to modify

    for x in range(grid_shape[0]):
        for y in range(grid_shape[1]):
            for z in range(grid_shape[2]):
                # Check if the current position is occupied
                if grid[x, y, z] == occupied_value:
                    # Check and update direct neighbors (6 neighbors)
                    neighbors = [(x - 1, y, z), (x + 1, y, z), (x, y - 1, z), (x, y + 1, z), (x, y, z - 1), (x, y, z + 1)]
                    for neighbor_x, neighbor_y, neighbor_z in neighbors:
                        if (
                            0 <= neighbor_x < grid_shape[0] and
                            0 <= neighbor_y < grid_shape[1] and
                            0 <= neighbor_z < grid_shape[2] and
                            grid[neighbor_x, neighbor_y, neighbor_z] != occupied_value
                        ):
                            new_grid[neighbor_x, neighbor_y, neighbor_z] = layer_value

    return new_grid

layer=small
for i in range(4):
    layer = add_layer_to_directly_occupied_neighbors(layer, 1, 1)
plot_empty_grid(layer)

In [239]:
def add_layer_within_distance(grid, layer_value, radius):
    """
    Add a layer to a 3D grid at positions where neighbors are within a specified distance of occupied cells.

    Parameters:
        grid (numpy.ndarray): The 3D grid to which the layer will be added.
        occupied_value: The value that indicates an occupied position in the grid.
        layer_value: The value to set at positions where neighbors are within distance r of occupied cells.
        r (int): The maximum distance for a neighbor to be considered within range.

    Returns:
        numpy.ndarray: The modified 3D grid with the added layer.
    """
 
    grid_shape = grid.shape
    new_grid = np.copy(grid)  # Create a copy to modify
    points  = np.argwhere(new_grid!=0)

    for point in points:
                    x, y, z = point
                    # Iterate over all positions in the grid and check their Euclidean distance
                    for i in range(-radius, radius):
                        for j in range(-radius,radius):
                            for k in range(-radius, radius):
                                
                                if (0 <= x-i < grid_shape[0] and
                                    0 <= y-j < grid_shape[1] and
                                    0 <= z-k < grid_shape[2] ) :
                                    distance = np.sqrt((i)**2 +  (j)**2 + ( k)**2)
                                    if distance <= radius:
                                        new_grid[x-i, y-j, z-k] = layer_value

    return new_grid


layer=small
for i in range(2):
    layer = add_layer_within_distance(layer, 1, 3)
plot_empty_grid(layer)

In [ ]:
small_sub = find_smallest_bounding_grid(sub_grid[10])
subgrid = create_subgrid((0,0,0), np.shape(small))
unocupied = extract_unoccupied_shape(rectangle, subgrid, position=(30,61,56))
can_fit_shape_in_unoccupied_shape(small, unocupied)
